# Summarization of financial data using a Large Language Model (LLM)

This notebook aims to provide an introduction to documenting an LLM using the ValidMind Developer Framework. The use case presented is a summarization of financial news (https://huggingface.co/datasets/cnn_dailymail).

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate documentation about the data and model

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install -q validmind

Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "...",
  project = "..."
)

2023-10-09 16:33:04,815 - INFO(validmind.api_client): Connected to ValidMind. Project: [12] Credit Risk Scorecard - Initial Validation (clmotr3oa000aesy6c54wqb6g)


## Load data

### CNN dataset

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [3]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')

train_df = cnn_dataset.data['train'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()

train_df = train_df[['article','highlights']]
test_df = test_df[['article','highlights']]

train_df = train_df.head(5)
test_df = test_df.head(5)

## Model selection

In [4]:
from validmind.models import FoundationModel, Prompt
import os
import dotenv
import openai

dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")


def call_model(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message["content"]

## Prompt engineering

In [5]:
prompt_template = """
You are an AI with expertise in summarizing financial news. 
Your task is to provide a concise summary of the specific news article provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the article.

Article to Summarize:

```
{article}
```

Please respond with a concise summary of the article's main points.
Ensure that your summary is based on the content of the article and not on external information or assumptions.
""".strip()

prompt_variables = ["article"]

## Initialize ValidMind datasets and models

In [6]:
vm_test_ds = vm.init_dataset(
    dataset=test_df,
    text_column="article",
    target_column="highlights",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    test_ds=vm_test_ds,
)

2023-10-09 16:33:07,740 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-10-09 16:33:07,751 - INFO(validmind.models.foundation): Running predict() for `test_ds`... This may take a while


In [10]:
metric = vm.tests.run_test(
    "validmind.model_validation.ToxicityHistogram", 
    model=vm_model)

Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


In [11]:
metric = vm.tests.run_test(
    "validmind.model_validation.ToxicityScore", 
    model=vm_model)

Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


In [12]:
metric = vm.tests.run_test(
    "validmind.model_validation.RegardScore", 
    model=vm_model)

In [13]:
metric = vm.tests.run_test(
    "validmind.model_validation.RegardHistogram", 
    model=vm_model)